In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string

In [2]:
!pip install anvil-uplink

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.1/90.1 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ws4py: filename=ws4py-0.5.1-py3-none-any.whl size=45228 sha256=b2c55adf11335a709d5437d398b42d14486b795a939174134db8b133786c4cf6
  Stored in directory: /root/.cache/pip/wheels/2e/7c/ad/d9c746276bf024d44296340869fcb169f1e5d80fb147351a57
Successfully built ws4py


In [3]:
import anvil.server

In [4]:
anvil.server.connect("server_K6A4S73AVG7SQCJSDY3XUXYX-VW4XUDVM62PV65QT")    #("your-uplink-key")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default Environment" as SERVER


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
df_fake = pd.read_csv("/content/drive/MyDrive/pw4/Fake_news/Fake.csv")
#df_fake.to_csv("/content/Fake.csv")
#df_fake = pd.read_csv("/content/Fake.csv")
df_fake.head(4)

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"


In [7]:
df_true = pd.read_csv("/content/drive/MyDrive/pw4/Fake_news/True.csv")
#df_true.to_csv("/content/True.csv")
#df_true = pd.read_csv("/content/True.csv")
df_true.head(4)

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"


In [ ]:
#df_fake = pd.read_csv("/content/Fake.csv")
#df_true = pd.read_csv("/content/True.csv")


In [8]:
df_fake["class"] = 0
df_true["class"] = 1

print(df_fake.shape)
print(df_true.shape)
type(df_fake)

(23481, 5)
(21417, 5)


pandas.core.frame.DataFrame

In [9]:
num_fake_news , temp = df_fake.shape
num_true_news , temp = df_true.shape
print(num_true_news)
print(num_fake_news-1)
print(num_fake_news-11)
df_fake.shape

21417
23480
23470


(23481, 5)

###Partitioning training and test datasets

In [10]:
#axis=0 => rows and axis=1 => columns
df_fake_manual_testing = df_fake.tail(10)
for i in range(num_fake_news-1,num_fake_news-11,-1):
    df_fake.drop([i], axis = 0, inplace = True)
df_true_manual_testing = df_true.tail(10)
for i in range(num_true_news-1,num_true_news-11,-1):
    df_true.drop([i], axis = 0, inplace = True)


In [11]:
df_fake.head(5)

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [12]:
print(df_fake_manual_testing.shape)
print(df_true_manual_testing.shape)

(10, 5)
(10, 5)


### Merge fake and true manual testing dataframes

In [13]:
df_manual_testing = pd.concat([df_fake_manual_testing,df_true_manual_testing], axis= 0)
df_manual_testing.to_csv("manual_testing.csv")

### Merging traing dataframes

In [14]:
df_merge = pd.concat([df_true, df_fake],axis=0);
df_merge.shape

(44878, 5)

### Drop unwanted columns

In [15]:
df_merge.columns

Index(['title', 'text', 'subject', 'date', 'class'], dtype='object')

In [16]:
df_test = df_merge.drop(["title","subject","date"], axis=1)
df_test.shape

(44878, 2)

In [17]:
df_test.isnull().sum()

text     0
class    0
dtype: int64

### Shuffling  df_test

In [18]:
df_test = df_test.sample(frac=1)
df_test.head(4)

,text,class
17786,WASHINGTON (Reuters) - A U.S. Navy destroyer s...,1
8866,"Yesterday, President Barack Obama made a touch...",0
17910,VIENNA (Reuters) - Austria s far right Freedom...,1
15564,A different set of laws apply to illegals in f...,0


In [19]:
### drop index
df_test.reset_index(inplace=False)
df_test.head(5)


,text,class
17786,WASHINGTON (Reuters) - A U.S. Navy destroyer s...,1
8866,"Yesterday, President Barack Obama made a touch...",0
17910,VIENNA (Reuters) - Austria s far right Freedom...,1
15564,A different set of laws apply to illegals in f...,0
719,WASHINGTON (Reuters) - U.S. Representative Bob...,1


In [20]:
#df_test.drop(["index"], axis = 1 , inplace=True)
df_test.head(4)

,text,class
17786,WASHINGTON (Reuters) - A U.S. Navy destroyer s...,1
8866,"Yesterday, President Barack Obama made a touch...",0
17910,VIENNA (Reuters) - Austria s far right Freedom...,1
15564,A different set of laws apply to illegals in f...,0


#### Creating a function to convert the text in lowercase, remove the extra space, special chr., ulr and links.

In [21]:
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [22]:
df_test["text"] = df_test["text"].apply(wordopt)

### Creating DEpending and Independent variables

In [23]:
x = df_test["text"]
y = df_test["class"]

In [24]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

## 1. Logistic Regression

In [26]:
from sklearn.linear_model import LogisticRegression

In [27]:
LR = LogisticRegression()
LR.fit(xv_train,y_train)


LogisticRegression()

In [28]:

pred_lr=LR.predict(xv_test)

In [29]:
LR.score(xv_test, y_test)

0.9877896613190731

In [30]:
print(classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5802
           1       0.99      0.99      0.99      5418

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



### 2. Decision Tree Classification

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
DT = DecisionTreeClassifier()
DT.fit(xv_train, y_train)

DecisionTreeClassifier()

In [ ]:
pred_dt = DT.predict(xv_test)

In [ ]:
DT.score(xv_test, y_test)

0.9955436720142602

In [ ]:
print(classification_report(y_test, pred_dt))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5774
           1       1.00      1.00      1.00      5446

    accuracy                           1.00     11220
   macro avg       1.00      1.00      1.00     11220
weighted avg       1.00      1.00      1.00     11220



### 3. Gradient Boosting Classifier

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
GBC = GradientBoostingClassifier(random_state=0)
GBC.fit(xv_train, y_train)

GradientBoostingClassifier(random_state=0)

In [ ]:
pred_gbc = GBC.predict(xv_test)

In [ ]:
GBC.score(xv_test, y_test)

0.9953654188948307

In [ ]:
print(classification_report(y_test, pred_gbc))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00      5774
           1       0.99      1.00      1.00      5446

    accuracy                           1.00     11220
   macro avg       1.00      1.00      1.00     11220
weighted avg       1.00      1.00      1.00     11220



### 4. Random Forest Classifier

In [ ]:
#from sklearn.ensemble import RandomForestClassifier

In [ ]:
#RFC = RandomForestClassifier(random_state=0)
#RFC.fit(xv_train, y_train)

In [ ]:
#pred_rfc = RFC.predict(xv_test)

In [ ]:
#RFC.score(xv_test, y_test)

In [ ]:
#print(classification_report(y_test, pred_rfc))

# Model Testing With Manual Entry

### News

In [31]:
def output_lable(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "Not A Fake News"

def manual_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(wordopt)
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)
    pred_LR = LR.predict(new_xv_test)
    pred_DT = DT.predict(new_xv_test)
    pred_GBC = GBC.predict(new_xv_test)
    #pred_RFC = RFC.predict(new_xv_test)

    print("\n\nLR Prediction: {} \nDT Prediction: {} \nGBC Prediction: {}".format(output_lable(pred_LR[0]), output_lable(pred_DT[0]), output_lable(pred_GBC[0]) ))
    #,
     #                                                                                                         output_lable(pred_GBC[0]),
      #   \nGBC Prediction: {} \nRFC Prediction: {}                                                                                                     output_lable(pred_RFC[0])


In [ ]:
@anvil.server.callable
def webSite_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(wordopt)
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)
    pred_LR = LR.predict(new_xv_test)
    print(pred_LR)
    # pred_DT = DT.predict(new_xv_test)
    return pred_LR[0]

def manual_testing_website(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(wordopt)
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)
    pred_LR = LR.predict(new_xv_test)
    pred_DT = DT.predict(new_xv_test)
    pred_GBC = GBC.predict(new_xv_test)
    #pred_RFC = RFC.predict(new_xv_test)

    return pred_LR +"\n"+ pred_DT +"\n"+ pred_GBC

anvil.server.wait_forever()

[1]
[1]
[1]
[0]


In [ ]:
news = str(input())
manual_testing(news)

In [ ]:
#news = str(input())
#manual_testing(news)

Paul Craig RobertsIn the last years of the 20th century fraud entered US foreign policy in a new way.  On false pretenses Washington dismantled Yugoslavia and Serbia in order to advance an undeclared agenda. In the 21st century this fraud multiplied many times. Afghanistan, Iraq, Somalia, and Libya were destroyed, and Iran and Syria would also have been destroyed if the President of Russia had not prevented it.  Washington is also behind the current destruction of Yemen, and Washington has enabled and financed the Israeli destruction of Palestine.  Additionally, Washington operated militarily within Pakistan without declaring war, murdering many women, children, and village elders under the guise of  combating terrorism.  Washington s war crimes rival those of any country in history.I have documented these crimes in my columns and books (Clarity Press). Anyone who still believes in the purity of Washington s foreign policy is a lost soul  Russia and China now have a strategic alliance 